## Astra Schema

### Initialise Connection

In [5]:
from astra_db import AstraSession

session = AstraSession().session

## Schema

**Daily Table Columns**

* comm_date: the day (bucket) date in the format YYYYMMDD; there is a new partition for each day.
* comm_time: the time a message is written in the format HHMMSSssss; there is a new row for each write message time.
* comm_offset_time: the time offset in the same format as comm_time; use this in conjunction with comm_offset_count to reduce the number of rows read.
* comm_offset_count: the number of messages from the start of the day to comm_offset_time.

To save on space, we store the time the message was written in a custom format that is smaller than a timestamp.


**Weekly Table Coumns**
This table will store the number of messages sent on every day in a given year for a given activity.

* comm_date: the day date in the format YYYYMMDD; there is a new row for each day in the week.
* comm_count: the number of messages sent for the given comm_date day.

Rather than store each message sent in a day for an activity, we store the total number of messages sent in a day. 
This significantly reduces the number of rows stored and the amount of work the application needs to do to get 
the total number of messages sent for a certain time period.

#### Communications Table

##### New

In [ ]:
#
# Communications Table
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS communication_data (
        comm_id text,
        comm_datetime timestamp,
        request_date date,
        activity text,
        category text,
        channel text,
        customer_id text,
        PRIMARY KEY (comm_id)
        ) WITH default_time_to_live = 7776000;
    """
)

##### Old

In [ ]:
#
# Communications Table
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS communications (
    communication_id UUID, 
    comm_datetime TIMESTAMP,
    customer_id UUID, 
    channel TEXT, 
    category_group TEXT, 
    category TEXT, 
    activity_name TEXT, 
    PRIMARY KEY ((communication_id), comm_datetime) );
    """
)

#### Event by Day Table

In [ ]:
#
# EVENTS TABLE
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS events_by_day (
        cap_date int,
        cap_type text,
        cap_action text,
        PRIMARY KEY (cap_date, cap_type, cap_action)
    );
    """
)

#### PROFILECAP

##### New

In [ ]:
#
# Communications by Customer and Channel
#
# This table will store the number of messages sent on every day in a given year 
# for a given customer, channel, and month.
# The number of messages sent per day per customer channel is assumed to be less than 100. 
# - comm_month: the month a message is sent to the client channel in the format YYYYMM.
# - comm_day: the day in the month a message is sent to the client channel in the format DD.
# 90 Day TTL
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS profilecap_month (
        customer_id text,
        channel text,
        comm_month int,
        comm_day int,
        communication_id text,
        PRIMARY KEY ((customer_id, channel, comm_month), comm_day, communication_id)
    ) WITH CLUSTERING ORDER BY (comm_day DESC) AND default_time_to_live = 7776000;
    """
)

##### Old

In [ ]:
#
# Communications by Customer and Channel
#
# This table will store the number of messages sent in a day for a given customer_id and channel.
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS profilecap_day (
        customer_id text,
        channel text,
        comm_date int, 
        comm_time int, 
        comm_offset_time int static,
        comm_offset_count int static,
        PRIMARY KEY ((customer_id, channel, comm_date), comm_time)
    ) WITH CLUSTERING ORDER BY (comm_time DESC) AND default_time_to_live = 604800;
    """
)

#
# This table will store the number of messages sent on every day in a given year for a given customer_id and channel.
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS profilecap_week (
        customer_id text,
        channel text,
        comm_date int,
        comm_count int,
        PRIMARY KEY ((customer_id,channel), comm_date)
    ) WITH CLUSTERING ORDER BY (comm_date DESC) AND default_time_to_live = 7776000;
    """
)

#### ACTYCAP

In [ ]:
#
# Communications by Activity
#
# This table will store the number of messages sent in a day for a given activity.
# 7 day TTL
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS actycap_day (
        activity text,
        comm_date int, 
        comm_time int, 
        comm_offset_time int static,
        comm_offset_count int static,
        PRIMARY KEY ((activity, comm_date), comm_time)
    ) WITH CLUSTERING ORDER BY (comm_time DESC) AND default_time_to_live = 604800;
    """
)

#
# This table will store the number of messages sent on every day in a given year for a given activity.
# 90 day TTL
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS actycap_week (
        activity text,
        comm_date int,
        comm_count int,
        PRIMARY KEY ((activity), comm_date)
    ) WITH CLUSTERING ORDER BY (comm_date DESC) AND default_time_to_live = 7776000;
    """
)

#### PRTYCAP

In [9]:
#
# Communications by category_group and category
#
# This table will store the number of messages sent in a day for a given category_group and category.
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS prtycap_day (
        category_group TEXT,
        category TEXT,
        comm_date int, 
        comm_time int, 
        comm_offset_time int static,
        comm_offset_count int static,
        PRIMARY KEY ((category_group, category, comm_date), comm_time)
    ) WITH CLUSTERING ORDER BY (comm_time DESC) AND default_time_to_live = 604800;
    """
)

#
# This table will store the number of messages sent on every day in a given year for a given category_group and category.
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS prtycap_week (
        category_group TEXT,
        category TEXT,
        comm_date int,
        comm_count int,
        PRIMARY KEY ((category_group, category), comm_date)
    ) WITH CLUSTERING ORDER BY (comm_date DESC) AND default_time_to_live = 7776000;
    """
)

#### CHANNELCAP

In [10]:
#
# Communications by Channel
#
# This table will store the number of messages sent in a day for a given channel.
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS channelcap_day (
        channel text,
        comm_date int, 
        comm_time int, 
        comm_offset_time int static,
        comm_offset_count int static,
        PRIMARY KEY ((channel, comm_date), comm_time)
    ) WITH CLUSTERING ORDER BY (comm_time DESC) AND default_time_to_live = 604800;
    """
)

#
# This table will store the number of messages sent on every day in a given year for a given activity.
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS channelcap_week (
        channel text,
        comm_date int,
        comm_count int,
        PRIMARY KEY ((channel), comm_date)
    ) WITH CLUSTERING ORDER BY (comm_date DESC) AND default_time_to_live = 7776000;
    """
)

### DailyCounts Trigger

In [ ]:
session.execute(
    """
    CREATE TABLE IF NOT EXISTS dailycounts_trigger (
        trigger_datetime timestamp,
        comm_date int,
        PRIMARY KEY (trigger_datetime)
    )
    """
)

## Drop Schema

In [2]:
session.execute("DROP TABLE IF EXISTS communications")
session.execute("DROP TABLE IF EXISTS cap_state")

session.execute("DROP TABLE IF EXISTS profilecap_day")
session.execute("DROP TABLE IF EXISTS profilecap_week")
session.execute("DROP TABLE IF EXISTS profilecap_month")

session.execute("DROP TABLE IF EXISTS actycap_day")
session.execute("DROP TABLE IF EXISTS actycap_week")

session.execute("DROP TABLE IF EXISTS prtycap_day")
session.execute("DROP TABLE IF EXISTS prtycap_week")

session.execute("DROP TABLE IF EXISTS channelcap_day")
session.execute("DROP TABLE IF EXISTS channelcap_week")

## Cleanup

In [4]:
session.shutdown()